# View MED Tracking Predictions

Notebook per generare il video del Mediterraneo con le tile ground truth/predette e la traccia del centro ciclone prevista dal modello di tracking (`output/tracking_inference_predictions.csv`).


In [ ]:
%%html
<style>

.dataframe td {
    white-space: normal !important;
    word-wrap: break-word;
    max-width: 900px;
}
</style>


In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['PATH'] = './ffmpeg-7.0.2-amd64-static:' + os.environ.get('PATH', '')

from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 1500)
# pd.set_option('display.max_columns', 1500)
# pd.set_option('display.precision', 3)
# pd.set_option('display.float_format', '{:.3f}'.format)
# pd.set_option('display.width', 21000)

import matplotlib.pyplot as plt
import matplotlib
from IPython.display import HTML

from arguments import prepare_finetuning_args, prepare_tracking_args
from dataset.build_dataset import calc_tile_offsets, get_train_test_validation_df
from dataset.data_manager import make_validation_data_builder_from_manos_tracks, BuildTrackingDataset
from view_test_tiles import (
    make_animation_parallel_ffmpeg,
    filling_missing_tile,
    safe_literal_eval,
    PALETTE,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Impossibile caricare la libreria torch dynamo


In [2]:
args = prepare_finetuning_args()

input_dir = Path('../fromgcloud')
output_dir = Path('../airmassRGB/supervised/')
manos_csv = Path('medicane_data_input/medicanes_new_windows.csv')

#classification_csv = Path('output/inference_predictions.csv')
tracking_csv = Path('output/tracking_inference_predictions.csv')

#if not classification_csv.exists():
#    fallback = Path(args.output_dir) / classification_csv.name
#    if fallback.exists():
#        classification_csv = fallback
if not tracking_csv.exists():
    fallback = Path(args.output_dir) / tracking_csv.name
    if fallback.exists():
        tracking_csv = fallback

#assert classification_csv.exists(), f'Missing classification predictions at {classification_csv}'
assert tracking_csv.exists(), f'Missing tracking predictions at {tracking_csv}'

PALETTE['PRED'] = (255, 0, 0)

#print(f'Classification predictions: {classification_csv}')
print(f'Tracking predictions: {tracking_csv}')


Tracking predictions: output/tracking_inference_predictions.csv


In [3]:
args = prepare_tracking_args()
tracks_df = pd.read_csv(manos_csv, parse_dates=['time', 'start_time', 'end_time'])
tracks_df_train, tracks_df_test, tracks_df_val = get_train_test_validation_df(tracks_df, 0.7, args.val_split_fraction)
# sto prendendo solo il test: ho messo frac_val = 0.0
test_builder = BuildTrackingDataset(type='SUPERVISED', args=args)
test_builder.prepare_data(tracks_df_test, input_dir, output_dir)

df_predictions = pd.read_csv(tracking_csv)
df_joined = test_builder.df_video.merge(df_predictions, on='path', how='inner')

Cicloni nel train: 12, cicloni nel test: 6
Train rows: 564, Test rows: 382, Validation rows: 0
23 days 08:45:00
15 days 21:30:00
0 days 00:00:00
sorted_metadata_files num :  4516
Creo video senza salvarli...
1)  ->
Sbilanciato, ma comunque tengo traccia dei positivi e negativi
Con cicloni: 81 	 ids: [      0 7001715]
Senza cicloni: 819
900 video per il periodo (effettivo) da 2020-12-13 09:45:00 a 2020-12-16 13:00:00

2)  ->
Sbilanciato, ma comunque tengo traccia dei positivi e negativi
Con cicloni: 47 	 ids: [      0 7001716]
Senza cicloni: 505
552 video per il periodo (effettivo) da 2020-12-25 11:45:00 a 2020-12-27 10:00:00

3)  ->
Sbilanciato, ma comunque tengo traccia dei positivi e negativi
Con cicloni: 49 	 ids: [0]
Senza cicloni: 539
588 video per il periodo (effettivo) da 2021-10-28 11:45:00 a 2021-10-30 13:00:00

4)  ->
Sbilanciato, ma comunque tengo traccia dei positivi e negativi
Con cicloni: 18 	 ids: [0]
Senza cicloni: 198
216 video per il periodo (effettivo) da 2021-11-08 

In [4]:
df_predictions.path = df_predictions.path.str.split('/').str[-1]#.str[:-4].apply(safe_literal_eval)
df_predictions

,path,pred_x,pred_y,target_x,target_y,err_px,err_km,tile_offset_x,tile_offset_y,pred_x_global,pred_y_global,target_x_global,target_y_global,pred_lat,pred_lon,target_lat,target_lon
0,13-12-2020_1600_852_196,82.536575,104.073769,10.0,50.0,90.473903,295.468279,852.0,196.0,934.536575,300.073769,862.0,246.0,34.142723,24.075520,36.089121,21.863759
1,14-12-2020_0800_852_196,104.419716,49.579853,108.0,50.0,3.604852,13.226001,852.0,196.0,956.419716,245.579853,960.0,246.0,36.191333,25.275195,36.196281,25.422454
2,14-12-2020_2300_1065_196,24.380598,111.561554,24.0,112.0,0.580594,0.000000,1065.0,196.0,1089.380598,307.561554,1089.0,308.0,34.040715,29.580271,34.040715,29.580271
3,15-12-2020_1500_1065_196,128.712585,107.774254,127.0,108.0,1.727400,7.184869,1065.0,196.0,1193.712585,303.774254,1192.0,304.0,34.362588,33.624791,34.358920,33.546644
4,16-12-2020_0700_1065_196,161.107361,111.159447,158.0,106.0,6.022921,20.713466,1065.0,196.0,1226.107361,307.159447,1223.0,302.0,34.310723,34.844462,34.492571,34.795502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,26-12-2020_1600_426_0,157.951859,150.527496,161.0,148.0,3.959723,16.874820,426.0,0.0,583.951859,150.527496,587.0,148.0,39.653027,12.292039,39.776183,12.407316
156,28-10-2021_1700_639_196,58.075359,68.842407,59.0,70.0,1.481547,4.968464,639.0,196.0,697.075359,264.842407,698.0,266.0,35.257462,15.980397,35.220653,16.011411
157,29-10-2021_0900_639_196,56.973526,38.153591,55.0,37.0,2.285952,7.235171,639.0,196.0,695.973526,234.153591,694.0,233.0,36.426144,16.055283,36.463306,15.988887
158,30-10-2021_0100_639_196,70.676834,45.731743,70.0,44.0,1.859311,8.731822,639.0,196.0,709.676834,241.731743,709.0,240.0,36.129047,16.514727,36.204441,16.487526


In [5]:
test_builder.df_video

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,id_cyc_unico
0,0,0,0,13-12-2020_1100_0_0,0,2020-12-13 09:45:00,2020-12-13 11:00:00,"[../fromgcloud/airmass_rgb_20201213_0945.png, ...",0
1,1,0,0,13-12-2020_1200_0_0,0,2020-12-13 10:45:00,2020-12-13 12:00:00,"[../fromgcloud/airmass_rgb_20201213_1045.png, ...",0
2,2,0,0,13-12-2020_1300_0_0,0,2020-12-13 11:45:00,2020-12-13 13:00:00,"[../fromgcloud/airmass_rgb_20201213_1145.png, ...",0
3,3,0,0,13-12-2020_1400_0_0,0,2020-12-13 12:45:00,2020-12-13 14:00:00,"[../fromgcloud/airmass_rgb_20201213_1245.png, ...",0
4,4,0,0,13-12-2020_1500_0_0,0,2020-12-13 13:45:00,2020-12-13 15:00:00,"[../fromgcloud/airmass_rgb_20201213_1345.png, ...",0
...,...,...,...,...,...,...,...,...,...
1399,1399,1065,196,09-09-2023_2200_1065_196,0,2023-09-09 20:45:00,2023-09-09 22:00:00,"[../fromgcloud/airmass_rgb_20230909_2045.png, ...",0
1400,1400,1065,196,09-09-2023_2300_1065_196,0,2023-09-09 21:45:00,2023-09-09 23:00:00,"[../fromgcloud/airmass_rgb_20230909_2145.png, ...",0
1401,1401,1065,196,10-09-2023_0000_1065_196,0,2023-09-09 22:45:00,2023-09-10 00:00:00,"[../fromgcloud/airmass_rgb_20230909_2245.png, ...",0
1402,1402,1065,196,10-09-2023_0100_1065_196,0,2023-09-09 23:45:00,2023-09-10 01:00:00,"[../fromgcloud/airmass_rgb_20230909_2345.png, ...",0


In [11]:
df_joined

,video_id,tile_offset_x_x,tile_offset_y_x,path,label,start_time,end_time,orig_paths,id_cyc_unico,pred_x,...,tile_offset_x_y,tile_offset_y_y,pred_x_global,pred_y_global,target_x_global,target_y_global,pred_lat,pred_lon,target_lat,target_lon


In [7]:
records = []
for _, row in df_joined.iterrows():
    for orig_path in row['orig_paths']:
        records.append({
            'path': orig_path,
            'predictions': row['predictions'],
            'tmp_label': row['labels'],
            'tile_offset_x': row['tile_offset_x'],
            'tile_offset_y': row['tile_offset_y'],
            'neighboring': row['neighboring'],
        })

df_mapping = pd.DataFrame(records).drop_duplicates()
df_mapping



""


In [9]:

def expand_mediterranean_dataframe(df_mapping: pd.DataFrame, builder) -> pd.DataFrame:
    """Espande il dataframe in modo da avere tutte le tile per frame, segnando quelle mancanti."""

    df_data = (
        df_mapping
        .merge(builder.master_df, on=['path', 'tile_offset_x', 'tile_offset_y'], how='left')
        .drop(columns='label')
        .rename(columns={'tmp_label': 'label'})
    )

    offsets = calc_tile_offsets(stride_x=213, stride_y=196)
    df_offsets = pd.DataFrame(offsets, columns=['tile_offset_x', 'tile_offset_y'])

    def expand_group(group):
        merged = df_offsets.merge(group, on=['tile_offset_x', 'tile_offset_y'], how='left', indicator=True)
        path_value = group['path'].iloc[0]
        merged['path'] = path_value

        for col in ['datetime']:
            val = group[col].iloc[0]
            merged[col] = merged[col].fillna(val)

        merged[filling_missing_tile] = merged['_merge'] == 'left_only'
        extra_cols = [c for c in group.columns if c not in ['path', 'tile_offset_x', 'tile_offset_y']]
        return merged[['path', 'tile_offset_x', 'tile_offset_y'] + extra_cols + [filling_missing_tile]]

    expanded = (
        df_data
        .groupby('path', group_keys=False)
        .apply(expand_group)
        .reset_index(drop=True)
    )

    expanded['predictions'] = expanded['predictions'].astype('Int8')
    expanded['label'] = expanded['label'].astype('Int8')
    return expanded, builder


In [10]:
expanded_df, val_builder = expand_mediterranean_dataframe(df_mapping, test_builder)

KeyError: 'path'

In [5]:
print(f"Frame totali: {expanded_df['path'].nunique()} immagini - {len(expanded_df)} righe complessive.")
expanded_df.head(2)

NameError: name 'expanded_df' is not defined

In [ ]:
def prepare_tracking_predictions(csv_path: Path, output_root: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df['path'] = df['path'].astype(str)

    root_str = str(output_root)
    df['path'] = df['path'].str.replace('/leonardo_scratch/fast/IscrC_SAME-D/airmass', root_str, regex=False)
    df['path'] = df['path'].str.replace('../airmassRGB/supervised', root_str, regex=False)

    df['tile_name'] = df['path'].apply(lambda p: Path(p).name)
    df['frame_datetime'] = pd.to_datetime(
        df['tile_name'].str.split('_').str[:2].str.join('_'),
        format='%d-%m-%Y_%H%M',
        errors='coerce'
    )

    df = df[~df['frame_datetime'].isna()].copy()
    df.sort_values('err_px', inplace=True)
    df_unique = df.drop_duplicates(subset='frame_datetime', keep='first').copy()

    df_unique = df_unique.rename(columns={
        'pred_x_global': 'track_pred_x',
        'pred_y_global': 'track_pred_y',
        'target_x_global': 'track_target_x',
        'target_y_global': 'track_target_y',
        'pred_lat': 'track_pred_lat',
        'pred_lon': 'track_pred_lon',
        'target_lat': 'track_target_lat',
        'target_lon': 'track_target_lon',
        'err_px': 'track_err_px',
        'err_km': 'track_err_km',
    })

    cols_to_keep = [
        'frame_datetime', 'track_pred_x', 'track_pred_y', 'track_target_x', 'track_target_y',
        'track_pred_lat', 'track_pred_lon', 'track_target_lat', 'track_target_lon',
        'track_err_px', 'track_err_km'
    ]

    return df_unique[cols_to_keep]


In [ ]:
tracking_df = prepare_tracking_predictions(tracking_csv, output_dir)
print(f'Tracking timestamp unici: {len(tracking_df)}')
tracking_df.head()


In [ ]:
df_with_tracks = expanded_df.merge(
    tracking_df,
    left_on='datetime',
    right_on='frame_datetime',
    how='left'
)
df_with_tracks.drop(columns=['frame_datetime'], inplace=True)
print(f'Righe complessive dopo merge: {len(df_with_tracks)}')


In [ ]:
def ensure_list(val):
    if isinstance(val, list):
        return list(val)
    if pd.isna(val):
        return []
    parsed = safe_literal_eval(val)
    if isinstance(parsed, list):
        return list(parsed)
    if pd.isna(parsed):
        return []
    return [parsed]

for col in ['x_pix', 'y_pix', 'source']:
    df_with_tracks[col] = df_with_tracks[col].apply(ensure_list)

def append_track_row(row):
    if pd.notna(row.get('track_pred_x')) and pd.notna(row.get('track_pred_y')):
        row['x_pix'] = row['x_pix'] + [float(row['track_pred_x'])]
        row['y_pix'] = row['y_pix'] + [float(row['track_pred_y'])]
        row['source'] = row['source'] + ['PRED']
    return row

df_with_tracks = df_with_tracks.apply(append_track_row, axis=1)


In [ ]:
tracked_only = df_with_tracks[df_with_tracks['track_pred_x'].notna()].copy()
tracked_only = tracked_only.sort_values('datetime').reset_index(drop=True)
print(f"Frame con track predette: {tracked_only['path'].nunique()} / {df_with_tracks['path'].nunique()}")
tracked_only[['path', 'datetime', 'track_pred_x', 'track_pred_y', 'track_err_px']].head()


In [ ]:
# Genera i frame e il video MP4 (richiede ffmpeg disponibile nel PATH)
make_animation_parallel_ffmpeg(
    tracked_only,
    output_folder='./anim_frames_tracking',
    nomefile='med_tracking_predictions'
)
